# 7. CNN

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-합성곱/풀링-계층-구현하기" data-toc-modified-id="1.-합성곱/풀링-계층-구현하기-1">1. 합성곱/풀링 계층 구현하기</a></span><ul class="toc-item"><li><span><a href="#1.1-4차원-배열" data-toc-modified-id="1.1-4차원-배열-1.1">1.1 4차원 배열</a></span></li><li><span><a href="#1.2-im2col로-데이터-전개하기" data-toc-modified-id="1.2-im2col로-데이터-전개하기-1.2">1.2 im2col로 데이터 전개하기</a></span></li><li><span><a href="#1.3-합성곱-계층-구현하기" data-toc-modified-id="1.3-합성곱-계층-구현하기-1.3">1.3 합성곱 계층 구현하기</a></span></li><li><span><a href="#1.4-풀링-계층-구현하기" data-toc-modified-id="1.4-풀링-계층-구현하기-1.4">1.4 풀링 계층 구현하기</a></span></li></ul></li><li><span><a href="#2.-CNN-구현하기" data-toc-modified-id="2.-CNN-구현하기-2">2. CNN 구현하기</a></span></li></ul></div>

## 1. 합성곱/풀링 계층 구현하기

### 1.1 4차원 배열

In [2]:
import numpy as np
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [6]:
print(x[0].shape)

(1, 28, 28)


In [7]:
x[0, 0]

array([[0.62484178, 0.61504766, 0.43111229, 0.63699605, 0.43174274,
        0.59899419, 0.20166019, 0.5845659 , 0.94309373, 0.93073241,
        0.59396344, 0.26429782, 0.54242127, 0.33554   , 0.91387033,
        0.25288109, 0.61075695, 0.05186686, 0.9960439 , 0.5499893 ,
        0.84014404, 0.49858534, 0.60589325, 0.13343462, 0.34027894,
        0.42848101, 0.43637676, 0.4953342 ],
       [0.37193873, 0.33835923, 0.86086126, 0.23174238, 0.02545073,
        0.45465432, 0.02882263, 0.82973089, 0.3837974 , 0.40873705,
        0.47492172, 0.23054144, 0.50059863, 0.6121458 , 0.98646823,
        0.95233409, 0.7262514 , 0.32600307, 0.59355836, 0.91151622,
        0.24915283, 0.76442376, 0.51214871, 0.21459938, 0.12488839,
        0.66838591, 0.8839296 , 0.41813027],
       [0.37056004, 0.2716555 , 0.16633975, 0.73302296, 0.79676365,
        0.89172179, 0.93958134, 0.59280796, 0.64640976, 0.28767714,
        0.96346624, 0.98225364, 0.98519837, 0.98170334, 0.68885663,
        0.53867701, 0.3200

### 1.2 im2col로 데이터 전개하기
* 합성곱 연산을 for문으로 구현하면
    * 시간도 오래 걸리고
    * numpy 원소 접근 시 for문을 쓰면 효율이 떨어진다고 한다.
* 이를 위해 im2col이라는 함수를 사용한다.
    * 4차원의 입력 데이터를 2차원 행렬로 전개하여 이를 계산하고,
    * 다시 4차원으로 reshape시켜 다음 계층으로 전달한다.
    * 4차원의 입력 데이터를 2차원으로 전개시키면
        * 합성곱에 의해 기존 데이터보다 많아지는데, 
        * 메모리는 더 많이 소비하지만, 컴퓨터 자체가 큰 행렬을 묶어 계산하는데 탁월하다는 장점이 있다.
        * 또한 선형대수 라이브러리들이 큰 행렬의 곱셈에 최적화있기 때문에 효율적이다.

### 1.3 합성곱 계층 구현하기

In [12]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [15]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H+2*self.pad - FH) / self.stride)
        out_w = int(1 + (W+2*self.pad - FW) / self.stride)
        
        # 중요!
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        # 여기까지!
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out
    

### 1.4 풀링 계층 구현하기

In [14]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H-self.pool_h) / self.stride)
        out_w = int(1 + (W-self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out
        

## 2. CNN 구현하기

In [16]:
class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]
